In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
%run dataframes.ipynb

In [2]:
# Import lemmatiseur
import fr_core_news_md
nlp = fr_core_news_md.load()

In [3]:
# Fichier stop word
fichier=open('stopwords-fr.txt','r')
stops=set()
for word in fichier:
    mot=re.sub('\n','',word)
    stops.add(mot)

In [43]:
# Fonction de normalisation
def normalize(text, stopwords):
    lemmatized = list()
    text = text.lower()
    text_nlp = nlp(text)
    for word in text_nlp:
        lemma = word.lemma_.strip()
        if lemma not in stopwords:
            lemmatized.append(lemma)
    return " ".join(lemmatized)

In [10]:
def normaliser(df, stops,support = 'titre'):
    dfN = df.copy()
    dfN[support] = [normalize(i,stops) for i in dft[support]]
    dfN['taille'] = [len(titre) for titre in dft[support]]
    return dfN

In [108]:
# La concaténation des lettres par semaine:
def concatener(df, support = 'titre'):
    dfC =  df.groupby(df.index).agg({ support: lambda x: ' '.join(x),'taille': lambda y:np.sum(y)}).rename({support:'concatenation'},axis=1)
    dfC['indice'] = dfC.taille.cumsum()
    return dfC

In [9]:
# Les scores du TF-IDF de tous les mots par semaine
def multi_scores(df):
    vectorizer = TfidfVectorizer()
    response=vectorizer.fit_transform(df.concatenation)
    feature_names = vectorizer.get_feature_names()
    dense = response.todense()
    denselist = dense.tolist()
    return pd.DataFrame(denselist, columns=feature_names).set_index(df.index)

In [137]:
# Fonction renvoyant le dataframe associé aux nb mots avec les plus grands scores par semaine
def premiers_scores(scores, nb):
    if nb == 0:
        return pd.concat([scores.idxmax(axis=1),scores.max(axis=1)],axis=1).rename(columns ={0:'mot',1:'score'})
    data = []
    for index, row in scores.iterrows():
        data.append(
            #[(mot, row[mot]) for mot in row.sort_values(ascending=False)[:5].index]
            [mot for mot in row.sort_values(ascending=False)[:nb].index]
        )
    return pd.DataFrame(data).set_index(scores.index)